# LightGBM from pickle
Pick up from RNAlight_122, which optimized LightGBM by RandomizedCV,
then saved the model to a pickle file, all using RNAlight code.    
Here, analyze and test the model.

In [1]:
import time
from datetime import datetime
print(datetime.now())

2024-02-20 14:19:18.416983


In [2]:
# Python import
import os
import copy
import random
import collections
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
from sklearn import svm
#from sklearn.externals import joblib
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [3]:
try:
    from google.colab import drive
    IN_COLAB = True
    print('Running on CoLab')
    PATH='/content/drive/'
    drive.mount(PATH)
    DATA_DIR=PATH+'My Drive/data/Localization/RNAlight/'  # must end in "/"
    MODEL_DIR=PATH+'My Drive/data/Localization/RNAlight/'  # must end in "/"
    output_dir=PATH+'My Drive/data/Localization/RNAlight/'
except:
    IN_COLAB = False
    DATA_DIR = './'    # Mac
    MODEL_DIR = './'    # Mac
    output_dir = './'
print('DATA DIR', DATA_DIR)

Running on CoLab
Mounted at /content/drive/
DATA DIR /content/drive/My Drive/data/Localization/RNAlight/


## Miniature version on Mac
The mac version was only optimized for 10 iterations of CV as a test.
See RNAlight_01_ML_lncRNA.mac.ipynb

In [4]:
if False:
  mac = 'mac.best_LightGBM_model.pkl'
  obj = joblib.load(mac)
  print(obj)
  vars(obj)

{'boosting_type': 'gbdt',
 'objective': 'binary',
 'num_leaves': 20,
 'max_depth': 40,
 'learning_rate': 0.02,
 'n_estimators': 400,
 'subsample_for_bin': 200000,
 'min_split_gain': 0.0,
 'min_child_weight': 0.001,
 'min_child_samples': 37,
 'subsample': 0.8,
 'subsample_freq': 1,
 'colsample_bytree': 0.6,
 'reg_alpha': 0.005,
 'reg_lambda': 0.001,
 'random_state': 100,
 'n_jobs': 1,
 'importance_type': 'split',
 '_Booster': <lightgbm.basic.Booster at 0x11a0d6250>,
 '_evals_result': {},
 '_best_score': defaultdict(collections.OrderedDict, {}),
 '_best_iteration': 0,
 '_other_params': {'subsample_freq': 1,
  'subsample': 0.8,
  'reg_lambda': 0.001,
  'reg_alpha': 0.005,
  'random_state': 100,
  'objective': 'binary',
  'num_leaves': 20,
  'n_jobs': 1,
  'n_estimators': 400,
  'min_child_samples': 37,
  'metric': 'binary_logloss',
  'max_depth': 40,
  'learning_rate': 0.02,
  'colsample_bytree': 0.6},
 '_objective': 'binary',
 'class_weight': None,
 '_class_weight': None,
 '_class_map': {0: 0, 1: 1},
 '_n_features': 1344,
 '_n_features_in': 1344,
 '_classes': array([0, 1]),
 '_n_classes': 2,
 'metric': 'binary_logloss',
 '_le': LabelEncoder(),
 'fitted_': True}

## Full version on CoLab
The colab version was optimized for the full 10000 iterations.

In [5]:
model_file = DATA_DIR+'best_LightGBM_model.pkl'
lgb_best_model = joblib.load(model_file)
print(lgb_best_model)

LGBMClassifier(colsample_bytree=0.5, learning_rate=0.01, max_depth=40,
               metric='binary_logloss', min_child_samples=9, n_estimators=2200,
               n_jobs=1, num_leaves=35, objective='binary', random_state=100,
               reg_alpha=0.005, reg_lambda=0, subsample=0.6, subsample_freq=1)


In [6]:
# vars(obj)

{'boosting_type': 'gbdt',
 'objective': 'binary',
 'num_leaves': 35,
 'max_depth': 40,
 'learning_rate': 0.01,
 'n_estimators': 2200,
 'subsample_for_bin': 200000,
 'min_split_gain': 0.0,
 'min_child_weight': 0.001,
 'min_child_samples': 9,
 'subsample': 0.6,
 'subsample_freq': 1,
 'colsample_bytree': 0.5,
 'reg_alpha': 0.005,
 'reg_lambda': 0,
 'random_state': 100,
 'n_jobs': 1,
 'importance_type': 'split',
 '_Booster': <lightgbm.basic.Booster at 0x79978967b340>,
 '_evals_result': {},
 '_best_score': defaultdict(collections.OrderedDict, {}),
 '_best_iteration': 0,
 '_other_params': {'metric': 'binary_logloss'},
 '_objective': 'binary',
 'class_weight': None,
 '_class_weight': None,
 '_class_map': {0: 0, 1: 1},
 '_n_features': 1344,
 '_n_features_in': 1344,
 '_classes': array([0, 1]),
 '_n_classes': 2,
 'metric': 'binary_logloss',
 '_le': LabelEncoder(),
 'fitted_': True}

In [7]:
hyperparams=lgb_best_model.get_params()
print(hyperparams)

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.5, 'importance_type': 'split', 'learning_rate': 0.01, 'max_depth': 40, 'min_child_samples': 9, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 2200, 'n_jobs': 1, 'num_leaves': 35, 'objective': 'binary', 'random_state': 100, 'reg_alpha': 0.005, 'reg_lambda': 0, 'subsample': 0.6, 'subsample_for_bin': 200000, 'subsample_freq': 1, 'metric': 'binary_logloss'}


## Cross-validation
The grid search is performed during 5FCV.
The best model is taken from the one fold with highest AUROC.
That model is not retrained on the entire training set.
Not possible for us to reproduce the 5FCV exactly because it relies on RandomizedSearchCV to make the train/valid splits.
Also, the LGBM API train() method takes number of iterations \(epochs\), which we don't know.

In [8]:
#Evaluate performance of model
def evaluate_performance(y_test, y_pred, y_prob):
    # AUROC
    auroc = metrics.roc_auc_score(y_test,y_prob)
    auroc_curve = metrics.roc_curve(y_test, y_prob)
    # AUPRC
    auprc=metrics.average_precision_score(y_test, y_prob)
    auprc_curve=metrics.precision_recall_curve(y_test, y_prob)
    #Accuracy
    accuracy=metrics.accuracy_score(y_test,y_pred)
    #MCC
    mcc=metrics.matthews_corrcoef(y_test,y_pred)

    recall=metrics.recall_score(y_test, y_pred)
    precision=metrics.precision_score(y_test, y_pred)
    f1=metrics.f1_score(y_test, y_pred)
    class_report=metrics.classification_report(y_test, y_pred,target_names = ["control","case"])

    model_perf = {"auroc":auroc,"auroc_curve":auroc_curve,
                  "auprc":auprc,"auprc_curve":auprc_curve,
                  "accuracy":accuracy, "mcc": mcc,
                  "recall":recall,"precision":precision,"f1":f1,
                  "class_report":class_report}

    return model_perf

In [9]:
# Count the frequency of k-mer in each RNA sequence
# k-mer was normalized by total k-mer count of each RNA sequence
def _count_kmer(Dataset,k): # k = 3,4,5

    # copy dataset
    dataset = copy.deepcopy(Dataset)
    # alphbet of nucleotide
    nucleotide = ['A','C','G','T']

    # generate k-mers
    #  k == 5:
    five = list(itertools.product(nucleotide,repeat=5))
    pentamer = []
    for n in five:
        pentamer.append("".join(n))

    #  k == 4:
    four = list(itertools.product(nucleotide,repeat=4))
    tetramer = []
    for n in four:
        tetramer.append("".join(n))

    # k == 3:
    three = list(itertools.product(nucleotide,repeat=3))
    threemer = []
    for n in three:
        threemer.append("".join(n))

    # input features can be combinations of diffrent k values
    if k == 34:
        table_kmer = dict.fromkeys(threemer,0)
        table_kmer.update(dict.fromkeys(tetramer,0))
    if k == 45:
        table_kmer = dict.fromkeys(tetramer,0)
        table_kmer.update(dict.fromkeys(pentamer,0))
    if k == 345:
        table_kmer = dict.fromkeys(threemer,0)
        table_kmer.update(dict.fromkeys(tetramer,0))
        table_kmer.update(dict.fromkeys(pentamer,0))

    # count k-mer for each sequence
    for mer in table_kmer.keys():
        table_kmer[mer] = dataset["cdna"].apply(lambda x : x.count(mer))

    # for k-mer raw count without normalization, index: nuc:1 or cyto:0
    rawcount_kmer_df = pd.DataFrame(table_kmer)
    df1_rawcount = pd.concat([rawcount_kmer_df,dataset["ensembl_transcript_id"]],axis = 1)
    df1_rawcount.index = dataset["tag"]

    # for k-mer frequency with normalization , index: nuc:1 or cyto:0
    freq_kmer_df = rawcount_kmer_df.apply(lambda x: x/x.sum(),axis=1)
    df1 = pd.concat([freq_kmer_df,dataset["ensembl_transcript_id"]],axis = 1)
    df1.index = dataset["tag"]

    return df1,df1_rawcount

In [10]:
SEED = 100

In [11]:
# load data
cyto_f = DATA_DIR+"02_lncRNA_info_cyto_transcript.tsv"
nuc_f =  DATA_DIR+"02_lncRNA_info_nuc_transcript.tsv"

dataset_cyto = pd.read_csv(cyto_f,sep='\t',index_col = False)    #1806
dataset_nuc = pd.read_csv(nuc_f,sep='\t',index_col = False)    #1986
# Set the tag of RCI(log2FC): nuclear 1 / cytosol 0
dataset_nuc['tag'] = 1;dataset_cyto['tag'] = 0
# merge the nuc and cyto dataset
dataset = pd.concat([dataset_nuc,dataset_cyto]) # 3792
# remove duplications(actually,each lncRNA is unique in its class)
dataset.drop_duplicates(keep="first",subset=["ensembl_transcript_id","name","cdna"],inplace=True) # 3792

# k = 3,4,5 count the normalized and raw count of kmer
df_kmer_345,df_kmer_345_rawcount = _count_kmer(dataset,345)
df_kmer_345.to_csv(os.path.join(output_dir,"df_kmer345_freq.tsv"),sep='\t')
df_kmer_345_rawcount.to_csv(os.path.join(output_dir,"df_kmer345_rawcount.tsv"),sep='\t')
# load kmer file
# df_kmer_345 = pd.read_csv(os.path.join(output_dir,"df_kmer345_freq.tsv"),sep='\t',index_col= 0)

# convert to x:kmer-freq , y:label
del df_kmer_345['ensembl_transcript_id']
x_kmer = df_kmer_345.values
y_kmer = y_kmer = np.array(df_kmer_345.index)
# split into training and test sets (9:1)
x_train, x_test, y_train, y_test = train_test_split(x_kmer, y_kmer, test_size = 0.1, random_state = SEED)


In [15]:
print('Train set balance:')
unique,counts = np.unique(y_train, return_counts=True)
print(unique)
print(counts)
print('Test set balance:')
unique,counts = np.unique(y_test, return_counts=True)
print(unique)
print(counts)

Train set balance:
[0 1]
[1622 1790]
Test set balance:
[0 1]
[184 196]


## Test set performance

In [16]:
print(datetime.now())
print('Test model')
#Get predict_class(y_pred) and predict_probality_for_case(y_prob) of TestSet
y_pred = lgb_best_model.predict(x_test)
y_prob = lgb_best_model.predict_proba(x_test)[:,1]
print('Predictions balance:')
unique,counts = np.unique(y_pred, return_counts=True)
print(unique)
print(counts)
print('Done')

2024-02-20 14:25:52.254964
Test model
Predictions balance:
[0 1]
[176 204]
Done


In [13]:
print('RNAlight statistics code')
model_perf = evaluate_performance(y_test,y_pred,y_prob)
for k,v in model_perf.items():
    if not k.endswith('_curve'):
        print(k, v)

auroc 0.7765361579414375
auprc 0.7733995967254652
accuracy 0.7052631578947368
mcc 0.40951635220331667
recall 0.7346938775510204
precision 0.7058823529411765
f1 0.7200000000000001
class_report               precision    recall  f1-score   support

     control       0.70      0.67      0.69       184
        case       0.71      0.73      0.72       196

    accuracy                           0.71       380
   macro avg       0.71      0.70      0.70       380
weighted avg       0.71      0.71      0.70       380



In [17]:
print('sklearn statistics code')
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix


sklearn statistics code


In [26]:
def show_perf(y_test,y_pred,y_prob):
    accuracy = accuracy_score(y_test, y_pred)*100.
    precision = precision_score(y_test, y_pred)*100.
    recall = recall_score(y_test, y_pred)*100.
    f1 = f1_score(y_test, y_pred)*100.
    mcc = matthews_corrcoef(y_test, y_pred)
    prc_Y, prc_X, prc_bins = precision_recall_curve(y_test, y_prob)
    auprc = auc(prc_X,prc_Y)*100.
    auroc = roc_auc_score(y_test, y_prob)*100.
    correct_ones = 0; correct_zeros = 0
    total_ones = 0; total_zeros = 0
    for i in range(len(y_pred)):
        if y_test[i]==1:
            total_ones += 1
            if y_pred[i]==1:
              correct_ones += 1
        if y_test[i]==0:
            total_zeros += 1
            if y_pred[i]==0:
                correct_zeros += 1
    sensitivity = (correct_ones/total_ones)*100.
    specificity = (correct_zeros/total_zeros)*100.

    print('accuracy:',accuracy,'\nprecision:',precision,'\nrecall:',recall,\
      '\nsensitivity:',sensitivity,'\nspecificity:',specificity,\
    '\nF1:',f1,'\nMCC:',mcc,'\nAUPRC:',auprc,'\nAUROC:',auroc)

In [27]:
show_perf(y_test,y_pred,y_prob)

accuracy: 70.52631578947368 
precision: 70.58823529411765 
recall: 73.46938775510205 
sensitivity: 73.46938775510205 
specificity: 67.3913043478261 
F1: 72.00000000000001 
MCC: 0.40951635220331667 
AUPRC: 77.2127857379498 
AUROC: 77.65361579414375


## Compare to RNAlight test
We used the RNAlight_01_ML_lncRNA notebook from the RNAlight repository.   
We modified it to print these values.   
Here, we confirm that predictions from our optimized model match those of the published optimized model.

In [34]:
y_pred_rep=[0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,1,0,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,0,0,0,0,1,0,1,0,1,1,1,0,0,0,0,1,0,0,1,1,0,1,1,1,1,0,0,1,0,1,1,0,0,1,1,1,0,1,1,1,1,1,0,1,0,0,1,0,1,1,1,0,0,1,1,0,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0,1,1,0,1,0,0,1,1,1,1,0,1,1,0,0,1,0,0,1,1,0,1,1,1,0,1,0,1,1,0,1,1,0,0,1,1,0,1,0,0,0,0,1,0,1,1,0,0,0,1,1,1,1,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,1,1,0,1,1,1,0,0,0,1,1,0,1,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,1,0,0,1,0,1,0,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,0,1,1,0,0,0,1,0,1,1,0,1,1,1,0,0,1,0,1,0]

if (y_pred == y_pred_rep).all():
    print('Same!')


Same!
